In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/train.csv')
train

In [ ]:
train = train.fillna(0)

In [ ]:
train.loc[train['class_name'] == 'No finding',['x_max', 'y_max']] = 1

In [ ]:
train

# FastRCNN 계열은 배경 (no finding) 의 정답값은 무조건 0 이되어야함.
# 여기 class_id 에선 14 번으로 되어있음 (NaN 값들 보면알수있음)

In [ ]:
train['class_id'] = train['class_id'] +1

train.loc[train['class_id'] == 15,'class_id'] = 0

In [ ]:
train

In [ ]:
train[train['image_id'] == '9a5094b2563a1ef3ff50dc5c7ff71345']

In [ ]:
train[train['image_id'] == 'ca7e72954550eeb610fe22bf0244b7fa']

In [ ]:
from torch.utils.data import Dataset, DataLoader
import pydicom

class VinBigDataset(Dataset):
    def __init__(self,dataframe, image_dir, transforms = None):
        super().__init__()
        self.image_ids = dataframe['image_id'].unique()
        self.dataframe = dataframe
        self.image_dir = image_dir
        self.transforms = transforms
        
    def __getitem__(self, index):
        image_id = self.image_ids[index]
        # records --> 직접 학습을 할때, bbox 들을 관리하고 학습을 할수있게 도와줌.
        # 아래는 index 가 정렬이 안되어있음 -> reset_index
        records = self.dataframe[self.dataframe['image_id'] == image_id].reset_index(drop = True)
        dicom = pydicom.dcmread(self.image_dir + image_id + '.dicom')
        image = dicom.pixel_array
        # dicom 파일은 사람들이 움직이거나 하면 y 절편이 움직임 or 기울기가 변하게됌 --> 밝기에 영향을 줌 
        
        intercept = dicom.RescaleIntercept if 'RescaleIntercept' in dicom else 0
        slope = dicom.RescaleSlope if 'RescaleSlope' in dicom else 1
        
        if slope != 1:
            image = slope * image
            image = image.astype(np.int16)
        
        image += np.int16(intercept)
        
        # 3채널로 변경, float32 는 메모리 줄이기위해 쓰는것
        image = np.stack([image, image, image]).astype(np.float32)
        # 표준화, dicom 파일은 경사 (y절편이) 제대로 안되어있으면 (ex) 사람이 움직이면) 사진의 밝기에 영향을줌 --> pixel 값의 최소값이 0 이아닐수도있음
        image = image - image.min()
        # 최대값으로 나눠서 0과1사이로 변환
        image = image / image.max()
        # 채널값을 뒤로 옮김 원랜 0,1,2 순
        image = image.transpose(1,2,0)
        
        # 특정 사진을 가져왔을때, 그게 만약에 0 번이라면 (no finding), 정상인 애들
        if records.loc[0, 'class_id'] == 0:
            # 3개를 가져오니깐 하나만 가져오게 하는코드
            records = records.loc[[0],:]
            
        boxes = records[['x_min', 'y_min', 'x_max', 'y_max']].values
        
        # 각각의 바운딩박스마다 정답값(label) 을 넣어주는것 
        labels = torch.tensor(records['class_id'].values,dtype = torch.int64)
        # 항상 key 값은 아래와 동일 (boxes, labels)
        target = {'boxes' : boxes, 'labels' : labels}
        if self.transforms:
            # 딕셔너리 target 은 위에 만드는데 왜 또 딕셔너리 sample 을 만드는지? :
            # target 은 학습하거나 예측할때 쓰이는 정답값 (bounding box 와 labels)
            # sample 은 augmentation (albumentation) 할때 쓰이는 정답값들. 항상 key 값은 아래와 동일
            sample = {'image' : image, 'bboxes' : boxes, 'labels' : labels}
            sample = self.transforms(**sample)
            # 딕셔너리 키값에 접근하는법
            image = sample['image']
            # 텐서형식으로 바뀐 애들 저장
            # boxes 를 위에서 미리 변경을 안해주고 augmentation 돌고나서 해주는이유 : 이미지가 뒤집히면서 bounding box 도 뒤집혀야하기때문
            target['boxes'] = torch.tensor(sample['bboxes'])
            # tensor 형식으로 이미 위에서 바꿨기때문에 필요없음 --> target['labels'] = sample['labels']
        return image, target
        
    
    def __len__(self):
        return len(self.image_ids)
        

In [ ]:
from albumentations.pytorch.transforms import ToTensorV2

import albumentations as al

In [ ]:
# 이미지 사이즈 동일하게 변경해줘야함
def get_train_transform():
    return al.Compose([al.Resize(height = 512, width = 512), al.Flip(0.5),ToTensorV2()], bbox_params ={'format' : 'pascal_voc', 'label_fields': ['labels']})

In [ ]:
def get_valid_transform():
    return al.Compose([al.Resize(height = 512, width = 512),ToTensorV2()], bbox_params ={'format' : 'pascal_voc', 'label_fields': ['labels']})

In [ ]:
train_dataset = VinBigDataset(train, '/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/train/', get_train_transform())

In [ ]:
# 나중에는 train 이 아니라 valid 가 들어감. train_test_split

valid_dataset = VinBigDataset(train, '/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/train/', get_valid_transform())

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size = 8, num_workers = 4, collate_fn = collate_fn)

In [ ]:
valid_dataloader = DataLoader(valid_dataset, batch_size = 8, num_workers = 4, collate_fn = collate_fn)

In [ ]:
train

# wheat dataset 은 이미지 사이즈가 다 같음. 이건 다 다름.

In [ ]:
import torchvision

In [ ]:
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# 맨마지막 출력층을 클래스 갯수로 (91개 --> 15개)
# 맨 마지막 줄도 15 * 4 --> 60 으로 바뀜

model.roi_heads.box_predictor = FastRCNNPredictor(1024, 15)




In [ ]:
import torch

device = torch.device('cuda')

model.to(device)

# requires_grad --> 학습을 할수 있는 층, 없는 층이 있다. true 인 애들만 가져올것. freeze 되어있는애들 제외.

params = [x for x in model.parameters() if x.requires_grad]
optimizer = torch.optim.Adam(params)

In [ ]:
# 모델마다 달라짐. FasterRCNN 에서 학습할때 Loss 를 어디에 저장하고, 어떻게 업데이트 할건지 정하는것.

class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            # Loss 에 대한 평균을 구한다는것. 1.0 곱하는것은 loss 가 float 으로 나와야 오류가 안나서
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        # 한번 학습을 했으면 새로운 epoch 이 시작되기전에 초기화
        self.current_total = 0.0
        self.iterations = 0.0

In [ ]:
import warnings

warnings.filterwarnings('ignore')

num_epochs = 2
iteration = 1

loss_hist = Averager()

# 학습시작

for epoch in range(num_epochs):
    # epoch 마다 초기화해야함
    loss_hist.reset()
    for images, targets in train_dataloader:
        images = [x.to(device) for x in images]
        # 오른쪽의 targets 은 8개. t 는 그중 하나. 왼쪽 for 문은 그 하나의 딕셔너리를 접근했을때 key 값과 value 값을 접근해야함
        targets = [{k: v.to(device) for k,v in t.items()} for t in targets]
        # 분류문제와는 달리 detection 은 여러개의 오류값이나옴
        loss_dict = model(images, targets)
        # 딕셔너리에 있는 values 함수이기 때문에 () 가 필요
        losses = sum(x for x in loss_dict.values())
        # 텐서처럼 [] 로 감싸고 있는데 .item 해서 숫자로만 가져옴
        loss_value = losses.item()
        loss_hist.send(loss_value)
        # 가중치 초기화, 항상 이 3줄이나옴
        optimizer.zero_grad()
        # backpropagation
        losses.backward()
        # w 값에 lr 을 곱해줘서 update 를 해줌
        optimizer.step()
        # verbose 옵션. loss 값도 봐야함
        if iteration % 1 == 0:
            print(f'Iteration :{iteration} , Loss : {loss_value}')
            
        iteration += 1
    print(f'Epoch : {epoch}, Loss : {loss_hist.value}')
    
    

torch.save(model.state_dict(), 'best.pth')
        
        



# Loss 값이 왔다갔다 심함 --> keras 와 다름
# Keras 도 원래 이렇게 나옴. 이런애들을 바로안보내고 옛날에 있던 loss 들이 계속 저장이 되면서 평균값으로 나옴. 


In [ ]:
# Valid 데이터셋을 가져와서 detection 잘맞는지 틀리는지 그림그려보기, 리더보드 제출안하고 하는것

# iter --> 데이터를 8장씩 가져옴
# next --> 이번거 가져옴

images , targets = next(iter(valid_dataloader))

images = [image.to(device) for image in images]

targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

boxes = targets[0]['boxes'].cpu().numpy()

# image 는 targets 와 다름. 그냥 image
sample = images[0].permute(1,2,0).cpu().numpy()

#예측하기

model.eval()

outputs = model(images)

cpu_device = torch.device("cpu")
outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]


In [ ]:
import matplotlib.pyplot as plt
import cv2

a, b = plt.subplots(1,1, figsize = (20,12))

for i in boxes:
    # (xmin, ymin)  , (xmax, ymax)
    cv2.rectangle(sample, (i[0], i[1]), (i[2], i[3]), (100,0,0), 2)

b.set_axis_off()
b.imshow(sample)